In [1]:
! nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
# Colab
! pip install fastai2 -q
! pip install pydicom -q
! pip install kornia -q

In [0]:
from fastai2.basics import *
from fastai2.callback.all import *
from fastai2.vision.all import *
from fastai2.medical.imaging import *

import pydicom

import pandas as pd

In [5]:
dataset2 = 'jesperdramsch/siim-acr-pneumothorax-segmentation-data'

# Google Colab
config_path = Path('/content/drive')
learner_path = config_path/"My Drive"
data_path_base = Path('/content/datasets/')

data_path2 = data_path_base/dataset2

from google.colab import drive

drive.mount(str(config_path))
os.environ['KAGGLE_CONFIG_DIR'] = f"{config_path}/My Drive/.kaggle"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!kaggle datasets download {dataset2} -p "{data_path2}" --unzip

100% 3.03G/3.03G [00:33<00:00, 49.1MB/s]
100% 3.03G/3.03G [00:33<00:00, 97.7MB/s]


In [0]:
data_path2.ls()

In [0]:
(data_path2/'dicom-images-train').ls()

In [0]:
items = get_dicom_files(data_path2/'dicom-images-train')
items

In [0]:
path = untar_data(URLs.SIIM_SMALL)
path.ls()

In [0]:
(path/'train').ls()

In [0]:
(path/'train/Pneumothorax').ls()

In [0]:
items = get_dicom_files(path/f"train/")
items

In [0]:
patient = 5555
xray_sample = dcmread(items[patient])
xray_sample.show(figsize=(12, 12))

In [0]:
xray_sample.as_dict()

In [0]:
xray_sample.pixels.shape

In [0]:
xray_sample.pixels.min(), xray_sample.pixels.max()

In [0]:
xray_sample.pixels[300:310, 400:410]

In [0]:
xray_sample.scaled_px[300:310, 400:410]

In [0]:
xray_sample.pixels.freqhist_bins()

In [0]:
dicom_windows

In [0]:
# xray_sample.pct_in_window??

In [0]:
xray_sample.pct_in_window(*dicom_windows.brain)

In [0]:
xray_sample.pct_in_window(*dicom_windows.lungs)

In [0]:
xray_sample.pct_in_window(*dicom_windows.mediastinum)

In [0]:
ims = xray_sample.hist_scaled(), uniform_blur2d(xray_sample.hist_scaled(),50)
show_images(ims, titles=('orig', 'blurred'), figsize=(12, 6))

In [0]:
ims = xray_sample.hist_scaled(), gauss_blur2d(xray_sample.hist_scaled(),50)
show_images(ims, titles=('orig', 'blurred'), figsize=(12, 6))

In [0]:
xray_sample.mask_from_blur??

In [0]:
wind = xray_sample.windowed(*dicom_windows.lungs)
mask1 = xray_sample.mask_from_blur(dicom_windows.lungs)
mask2 = xray_sample.mask_from_blur(dicom_windows.spine_soft)
mask3 = xray_sample.mask_from_blur(dicom_windows.liver)

_,ax = subplots(2,2, figsize=(12, 12))
ax = ax.reshape(-1)
show_image(wind, ax=ax[0])
show_image(wind, ax=ax[1])
show_image(wind, ax=ax[2])
show_image(wind, ax=ax[3])

show_image(mask1, alpha=0.5, cmap=plt.cm.Reds, ax=ax[1], title="lungs");
show_image(mask2, alpha=0.5, cmap=plt.cm.Reds, ax=ax[2], title="spine_soft");
show_image(mask3, alpha=0.5, cmap=plt.cm.Reds, ax=ax[3], title="liver");

In [0]:
xray_sample.show(cmap=plt.cm.gist_ncar, figsize=(12, 12))

In [0]:
bbs = mask2bbox(mask)
bbs

In [0]:
lo,hi = bbs
show_image(wind[lo[0]:hi[0],lo[1]:hi[1]], figsize=(12, 12));

In [0]:
df = pd.read_csv(path/'labels.csv')
df.head()

In [0]:
pneumothorax = DataBlock(blocks=(ImageBlock(cls=PILDicom), CategoryBlock), 
                         get_x=lambda x: path/f"{x[0]}", 
                         get_y=lambda x: x[1], 
                         batch_tfms=[*aug_transforms(size=224)]
                         )

In [0]:
dls = pneumothorax.dataloaders(df.values)
dls.show_batch(max_n=16)

In [0]:
learn = cnn_learner(dls, resnet34, metrics=accuracy, cbs=[ShowGraphCallback])

In [0]:
learn.fine_tune(6)

# Credit

* https://dev.fast.ai/medical.imaging
* https://www.kaggle.com/jhoward/don-t-see-like-a-radiologist-fastai
* https://www.kaggle.com/jhoward/some-dicom-gotchas-to-be-aware-of-fastai
* https://www.kaggle.com/jesperdramsch/siim-acr-pneumothorax-segmentation-data
* https://github.com/fastai/fastai2/blob/master/nbs/61_tutorial.medical_imaging.ipynb
* 